In [125]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [126]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Demo for top 5 Document URL's based on document embedding

In [127]:
import gradio as gr
from transformers import AutoTokenizer, AutoModel
import pickle
import pandas as pd

model_ckpt = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

embeddings_dataset = pickle.load(open('Distillbert_DocumentLevel_pkl', 'rb'))

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

def semantic_search(query):
    question_embedding = get_embeddings([query]).cpu().detach().numpy()
    scores, samples = embeddings_dataset.get_nearest_examples(
        "Embeddings", question_embedding, k=5
    )
    samples_df = pd.DataFrame.from_dict(samples)
    samples_df["scores"] = scores
    samples_df.sort_values("scores", ascending=False, inplace=True)
    results = []
    for _, row in samples_df.iterrows():
        results.append(f" \nTITLE: {row.Title}" + f" \nURL: {row.URL} \n{'*' * 100}")
    return "".join(results)

Generating the GUI for search

In [128]:
import gradio as gr
import warnings
warnings.filterwarnings("ignore")

demo = gr.Blocks()

with demo:
    gr.Markdown(
        """
        Ask a Question
        The bot will return top 5 URL's related to the query given in input
"""
    )
    query_input = gr.inputs.Textbox(label='Query')
    result_output = gr.outputs.Textbox(label='Results')
    query_input.change(fn=semantic_search, inputs=query_input, outputs=result_output)


demo.launch()



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>